<h1><b>Importing Libraries</b></h1>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from collections import Counter
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import recall_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
#from imblearn.metrics import geometric_mean_score
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torchvision import datasets

import warnings
warnings.filterwarnings('ignore')

<h1><b>Dataset Loading</b></h1>

In [ ]:
dataset_path = '/kaggle/input/50-skin-disease/Best_50_class' 
dataset = datasets.ImageFolder(root= dataset_path)
class_names = dataset.classes
print(class_names)

In [ ]:
# dataset.targets has numeric labels for each image
counts = Counter(dataset.targets)

# Map counts to class names
for class_idx, count in counts.items():
    print(f"{dataset.classes[class_idx]}: {count} images")

In [ ]:
df_counts = pd.DataFrame({
    "Class": [dataset.classes[idx] for idx in counts.keys()],
    "Count": [counts[idx] for idx in counts.keys()]
})

# Save to CSV
df_counts.to_csv("class_counts.csv", index=False)

print("Counts saved to class_counts.csv")

<h1><b>Inception-with Unfrozen Layers</b></h1>

<h2><b>Dataset Splitting</b></h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

# =========================
# Dataset creation
# =========================
data_dir = dataset_path  # your dataset folder
img_size = (299, 299)    # InceptionV3 expects 299x299
batch_size = 32

# =========================
# Step 1: 70% training set
# =========================
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.30,   # leave 30% for val+test
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# =========================
# Step 2: 30% val+test pool
# =========================
val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.30,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# =========================
# Step 3: Split val_test_ds into 15% val + 15% test
# =========================
val_test_size = val_test_ds.cardinality().numpy()
val_size = val_test_size // 2
test_size = val_test_size - val_size  # handles odd numbers safely

val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

# =========================
# Preprocessing function
# =========================
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_batch(images, labels):
    images = preprocess_input(images)  # Apply InceptionV3 preprocessing
    return images, labels

train_ds = train_ds.map(preprocess_batch, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess_batch, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(preprocess_batch, num_parallel_calls=AUTOTUNE)

# Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
batch_size = 32  # Use the same batch size you set

def dataset_size(dataset):
    # Get number of batches
    batches = dataset.cardinality().numpy()
    if batches == tf.data.INFINITE_CARDINALITY or batches == tf.data.UNKNOWN_CARDINALITY:
        return "Unknown size"
    else:
        return batches * batch_size

print("Train set size:", dataset_size(train_ds))
print("Validation set size:", dataset_size(val_ds))
print("Test set size:", dataset_size(test_ds))

<h2><b>Model Compiling</b></h2>

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import layers, models, optimizers, callbacks

num_classes = train_ds.element_spec[1].shape[-1]

# 1) Base model
base = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
base.trainable = False

# 2) Classification head
inputs = layers.Input(shape=(299, 299, 3))
x = inputs
x = preprocess_input(x)  # if you didn’t map in the dataset pipeline
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.summary()

<h2><b>Model Training</b></h2>

In [ ]:
# 3) Warmup: train head only
model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
warmup_cb = [
    callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor="val_loss"),
    callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss"),
]

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Directory + filename to save best model
checkpoint_cb = ModelCheckpoint(
    "best_inception_model.h5",   # <-- file will be saved here
    save_best_only=True,         # save only the best model (lowest val_loss)
    monitor="val_loss",          # metric to monitor
    mode="min",                  # "min" because we want the lowest val_loss
    verbose=1
)

# --- Warmup training ---
history_1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True, monitor="val_loss"),
        ReduceLROnPlateau(patience=3, factor=0.5, monitor="val_loss"),
        checkpoint_cb  # <-- added
    ]
)

# Track how many epochs were actually trained
last_epoch = len(history_1.history['loss'])

# --- Unfreeze last Inception blocks ---
for layer in base.layers:
    layer.trainable = False
unfreeze = False
for layer in base.layers:
    if layer.name in ["mixed9", "mixed10"]:
        unfreeze = True
    if unfreeze:
        layer.trainable = True

# Re-compile with lower learning rate
model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# --- Fine-tuning training ---
history_2 = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=last_epoch,
    epochs=last_epoch + 50,   # extend training
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss"),
        ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss"),
        checkpoint_cb  # <-- added
    ]
)


<h2><b>Results</b></h2>

In [ ]:
acc = history_1.history['accuracy'] + history_2.history['accuracy'] 
val_acc = history_1.history['val_accuracy'] + history_2.history['val_accuracy'] 

loss = history_1.history['loss'] + history_2.history['loss'] 
val_loss = history_1.history['val_loss'] + history_2.history['val_loss'] 

epochs = range(1, len(acc) + 1)

In [ ]:
import pandas as pd

# Combine histories
acc = history_1.history['accuracy'] + history_2.history['accuracy']
val_acc = history_1.history['val_accuracy'] + history_2.history['val_accuracy']
loss = history_1.history['loss'] + history_2.history['loss']
val_loss = history_1.history['val_loss'] + history_2.history['val_loss']

epochs = range(1, len(acc) + 1)

# Create DataFrame
history_df = pd.DataFrame({
    "epoch": epochs,
    "accuracy": acc,
    "val_accuracy": val_acc,
    "loss": loss,
    "val_loss": val_loss
})

# Save to CSV
history_df.to_csv("combined_history_inceptionV3.csv", index=False)

print("✅ Training history saved to combined_history.csv")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 6))

num_epochs = epochs

# X ticks positions (integers from 0 to num_epochs, step 4)
xticks = range(0, len(epochs) + 2, 4)

# Plot accuracy
axs[0].plot(epochs, acc, label='Train Accuracy')
axs[0].plot(epochs, val_acc, label='Validation Accuracy')
axs[0].set_xlabel('Epoch', fontsize=20)
axs[0].set_ylabel('Accuracy', fontsize=20)
axs[0].set_title('Training and Validation Accuracy - InceptionV3', fontsize=22)
axs[0].legend(fontsize=18)
axs[0].set_xticks(xticks)
axs[0].set_yticks(np.arange(0.2, 1.0, 0.1))
axs[0].tick_params(axis='x', labelsize=18)
axs[0].tick_params(axis='y', labelsize=18)
axs[0].grid(True)

# Plot loss
axs[1].plot(epochs, loss, label='Train Loss')
axs[1].plot(epochs, val_loss, label='Validation Loss')
axs[1].set_xlabel('Epoch', fontsize=20)
axs[1].set_ylabel('Loss', fontsize=20)
axs[1].set_title('Training and Validation Loss - InceptionV3', fontsize=22)
axs[1].legend(fontsize=18)
axs[1].set_xticks(xticks)
axs[1].set_yticks(np.arange(0.5,2.75,0.25))
axs[1].tick_params(axis='x', labelsize=18)
axs[1].tick_params(axis='y', labelsize=18)
axs[1].grid(True)

plt.tight_layout()

# Save the combined figure
plt.savefig('InceptionV3_training_curves.png', dpi=600)
plt.savefig('InceptionV3_training_curves.pdf')

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef

# 1. Get true and predicted labels
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 2. Compute confusion matrix and metrics
cm = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f'Matthews Correlation Coefficient: {mcc:.4f}')

# 3. Plot confusion matrix
plt.figure(figsize=(30, 25))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels= class_names, yticklabels= class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('InceptionV3_confusion_matrix.png', dpi=600)
plt.savefig('InceptionV3_confusion_matrix.pdf')
plt.show()

# 4. Save metrics to CSV
metrics = {
    'Accuracy': [accuracy],
    'Recall': [recall],
    'Precision': [precision],
    'F1 Score': [f1],
    'MCC':[mcc]
}

# Save confusion matrix as CSV
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
df_cm.to_csv("confusion_inceptionv3.csv")

df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('InceptionV3_performance_metrics.csv', index=False)

<h1><b>Zipping all Files</b></h1>

In [ ]:
!zip -r /kaggle/working/output_files_inceptionv3_trainable.zip /kaggle/working/*